In [ ]:
# t3_optionA_compstat.py — T=3 with exclusivity in both remaining periods (2 & 3)
# Saves:
#   figures/t3A_think_vs_c.png
#   figures/t3A_think_vs_r.png
#   figures/t3A_vs_t2_think_vs_r.png
#   results/t3A_summary.csv

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
# Create output folders for figures and CSV summaries (idempotent)
os.makedirs("figures", exist_ok=True); os.makedirs("results", exist_ok=True)

# ---------------- Baseline ----------------
# Game primitives:
# p: prior that LEFT arm is good; s: success prob if the good arm is pulled; V: prize value.
# c0, r0: baseline thinking cost and breakthrough probability used for the comparative statics.
p, s, V = 0.6, 0.7, 1.0
c0, r0  = 0.10, 0.40

def L(p, s, V, r):
    """One-period continuation value for a right-holder: Λ = s*V*[ r + (1 - r)*p ]."""
    return s * V * (r + (1 - r) * p)

# ------------- T=3 Option A (exclusivity in periods 2 & 3), one decision at t=1 -------------
# Payoffs against an opponent who plays Do with probability x:
#   U_D^A(x) = p*s*V * (1 + x/2)              # immediate expected payoff + contest effect when both do
#   U_T^A(x) = -c + (1 + x) * L(p,s,V,r)      # pay cost now, earn Λ in both remaining periods with priority weight
# Indifference: c = (L - p*s*V) + x * (L - 0.5*p*s*V)
# -> x* = (c - Δ) / (L - 0.5*p*s*V),  with Δ ≡ L - p*s*V
def xstar_T3_optionA(p, s, V, c, r):
    lam   = L(p, s, V, r)          # Λ evaluated at (p,s,V,r)
    flat  = p * s * V              # baseline one-shot DO payoff with prior p
    delta = lam - flat             # Δ = informational surplus over flat DO
    denom = lam - 0.5 * flat       # denominator from indifference slope in x
    if c <= delta:                      # Always-Think region (cost low enough)
        return 0.0, 1.0, "always-think"
    c_upper = delta + denom            # Upper cutoff: c ≥ 2Λ - 1.5*flat -> Always-Do
    if c >= c_upper:                   # Always-Do region (cost high enough)
        return 1.0, 0.0, "always-do"
    x = (c - delta) / denom            # Interior mixing: solve for x*
    x = float(np.clip(x, 0.0, 1.0))    # clip for numerical safety
    return x, 1.0 - x, "interior"

# ------------- T=2 baseline (for overlay) -------------
def xstar_T2(p, s, V, c, r):
    """T=2 closed-form: returns (x_do, 1-x). Used only for overlay comparisons."""
    lam  = L(p, s, V, r)
    flat = p * s * V
    if (flat + c) <= 0.5 * lam: return 0.0, 1.0          # Always-Think
    if (flat + c) >= lam:       return 1.0, 0.0          # Always-Do
    x = 2.0 * (flat + c) / lam - 1.0                     # Interior mix
    x = float(np.clip(x, 0.0, 1.0))
    return x, 1.0 - x

# ================= Plot 1: Think vs cost (r fixed) =================
# Sweep c with r held at r0 to show how the T=3 (Option A) Think frequency reacts.
lam0  = L(p, s, V, r0); flat = p * s * V
c_lo  = lam0 - flat                   # Always-Think cutoff (= Δ)
c_hi  = 2 * lam0 - 1.5 * flat         # Always-Do cutoff

c_grid  = np.linspace(0.0, 0.8, 321)
think_c = [xstar_T3_optionA(p, s, V, c, r0)[1] for c in c_grid]

fig, ax = plt.subplots(figsize=(7.6, 4.4), constrained_layout=True)
ax.plot(c_grid, think_c, label="T=3 (Option A)")

# Shading the regions in c-space using cutoffs c_lo and c_hi
if c_lo > 0.0: ax.axvspan(0.0, min(c_lo, 0.8), alpha=0.25, color="#d1e7dd", label="Always Think")
Lft, Rgt = max(0.0, c_lo), min(0.8, c_hi)
if Lft < Rgt: ax.axvspan(Lft, Rgt, alpha=0.25, color="#cfe2ff", label="Interior mix")
if c_hi < 0.8: ax.axvspan(max(0.0, c_hi), 0.8, alpha=0.25, color="#fde2e4", label="Always Do")

# Draw dashed cutoffs and the baseline point (c0, r0)
ax.axvline(c_lo, linestyle="--", color="k", lw=1)
ax.axvline(c_hi, linestyle="--", color="k", lw=1)
ax.scatter([c0], [xstar_T3_optionA(p, s, V, c0, r0)[1]], zorder=3, color="k")

ax.set_title("T=3 (Option A): Think frequency vs thinking cost (r fixed)")
ax.set_xlabel("Thinking cost (c)")
ax.set_ylabel("Think frequency (1 - x*)")

# Reserve right margin for legend outside the axes
fig.subplots_adjust(right=0.82)
ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)

fig.savefig("figures/t3A_think_vs_c.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# ================= Plot 2: Think vs r (c fixed) =================
# r cutoffs as functions of c (invert the cost thresholds):
def r_lower_fun(c,p,s,V): return c / (s*V*(1-p))                # From c = Δ(r) = s*V*(1-p)*r
def r_upper_fun(c,p,s,V): return (c/(s*V) - 0.5*p) / (2*(1-p))  # From c = 2Λ(r) - 1.5*p*s*V

r_grid  = np.linspace(0.01, 0.99, 321)
think_r = [xstar_T3_optionA(p, s, V, c0, r)[1] for r in r_grid]
r_lo    = r_lower_fun(c0, p, s, V)   # start of Always-Think
r_hi    = r_upper_fun(c0, p, s, V)   # end of Interior -> Always-Do

fig, ax = plt.subplots(figsize=(7.6, 4.4), constrained_layout=True)
ax.plot(r_grid, think_r, label="T=3 (Option A)")

# Region shading in r-space using r_hi (left boundary) and r_lo (right boundary)
if 0.0 < r_hi < 1.0: ax.axvspan(0.0, r_hi, alpha=0.25, color="#fde2e4", label="Always Do")
Lft, Rgt = max(0.0, r_hi), min(1.0, r_lo)
if Lft < Rgt: ax.axvspan(Lft, Rgt, alpha=0.25, color="#cfe2ff", label="Interior mix")
if 0.0 < r_lo < 1.0: ax.axvspan(r_lo, 1.0, alpha=0.25, color="#d1e7dd", label="Always Think")

# Dashed thresholds and baseline marker at (c0, r0)
if 0.0 < r_hi < 1.0: ax.axvline(r_hi, linestyle="--", color="k", lw=1)
if 0.0 < r_lo < 1.0: ax.axvline(r_lo, linestyle="--", color="k", lw=1)
ax.scatter([r0], [xstar_T3_optionA(p, s, V, c0, r0)[1]], zorder=3, color="k")

ax.set_title("T=3 (Option A): Think frequency vs breakthrough probability (c fixed)")
ax.set_xlabel("Breakthrough probability (r)")
ax.set_ylabel("Think frequency (1 - x*)")

fig.subplots_adjust(right=0.82)
ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)

fig.savefig("figures/t3A_think_vs_r.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# ================= Overlay vs T=2 (reference) =================
# Compare T=3 Option A against the T=2 baseline across r.
tf_T2 = [xstar_T2(p, s, V, c0, r)[1] for r in r_grid]

fig, ax = plt.subplots(figsize=(7.6, 4.4), constrained_layout=True)
ax.plot(r_grid, tf_T2, label="T=2 (baseline)")
ax.plot(r_grid, think_r, "--", label="T=3 (Option A)")

# Optional: show T=2 threshold r† and baseline dot (visual anchor)
r_thresh = c0 / (s*V*(1-p))          # T=2 upper boundary mapped to r
ax.axvline(r_thresh, linestyle=":", color="k", lw=1)
tf0 = xstar_T2(p, s, V, c0, r0)[1]
ax.scatter([r0], [tf0], color="k", zorder=3)

ax.set_xlabel("Breakthrough probability (r)")
ax.set_ylabel("Think frequency (1 - x*)")
ax.set_title("Period-1 Think: T=2 baseline vs T=3 (Option A)")

ax.legend()
fig.savefig("figures/t3A_vs_t2_think_vs_r.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# ================= CSV summary =================
# Record representative points: baseline (c0,r0) and the cost/r thresholds evaluated at the other parameter.
rows = []
for (lab, c, r) in [
    ("baseline", c0, r0),
    ("c_lower at r0", c_lo, r0),
    ("c_upper at r0", c_hi, r0),
    ("r_lower at c0", c0, r_lo),
    ("r_upper at c0", c0, r_hi),
]:
    x, tf, reg = xstar_T3_optionA(p, s, V, c, r)
    rows.append({"label": lab, "c": float(c), "r": float(r), "x_do": x, "think_freq": tf, "region": reg})
pd.DataFrame(rows).to_csv("results/t3A_summary.csv", index=False)

print("Saved: figures/t3A_think_vs_c.png, figures/t3A_think_vs_r.png, figures/t3A_vs_t2_think_vs_r.png, results/t3A_summary.csv")


C:\Users\a13hk\AppData\Local\Temp\ipykernel_25636\3968949768.py:76: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  fig.subplots_adjust(right=0.82)
C:\Users\a13hk\AppData\Local\Temp\ipykernel_25636\3968949768.py:109: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  fig.subplots_adjust(right=0.82)


Saved: figures/t3A_think_vs_c.png, figures/t3A_think_vs_r.png, figures/t3A_vs_t2_think_vs_r.png, results/t3A_summary.csv
